<h1 align="center"> Segmenting and Clustering Neighborhoods in Toronto </h1>

## Import Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


from bs4 import BeautifulSoup

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.08 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  17.01 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  30.30 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  38.38 MB/s
Libraries imported.


In [63]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [64]:
soup = BeautifulSoup(url,'lxml')

In [65]:
# print(soup.prettify())

In [66]:
#find table and and it cells

table = soup.find('table')

row_info = table.find_all('td')

In [67]:
print(type(row_info))

<class 'bs4.element.ResultSet'>


In [68]:
#columns for dataframe
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(row_info), 3):
    postcode.append(row_info[i].text.strip())
    borough.append(row_info[i+1].text.strip())
    neighbourhood.append(row_info[i+2].text.strip())

In [69]:
#create dataframe
df_to = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_to.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_to.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Ignore cells with a borough that is Not assigned

In [70]:
a = df_to.index[df_to['Borough'] == 'Not assigned'].tolist()
print("Dropped Index is:",a)

df_to.drop(df_to.index[a],inplace=True)

df_to.head()

Dropped Index is: [0, 1, 9, 13, 20, 21, 30, 36, 37, 45, 46, 50, 51, 52, 54, 55, 59, 60, 61, 73, 74, 75, 88, 89, 90, 104, 105, 106, 120, 121, 136, 137, 148, 149, 155, 161, 162, 167, 175, 181, 182, 188, 189, 190, 194, 195, 201, 202, 203, 204, 209, 210, 223, 224, 238, 239, 242, 243, 248, 249, 254, 255, 259, 260, 261, 262, 264, 265, 275, 276, 277, 278, 279, 280, 281, 282, 288]


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Combine rows which have same Borough and will be combined into one row with the neighborhoods separated with a comma,

In [71]:
df_to = df_to.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_to.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_to.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Find a row who have a borough but a Not assigned neighborhood,After that we will replace it with default Queen's Park.

In [72]:
df_to.index[df_to['Neighbourhood'] == 'Not assigned'].tolist()

[85]

In [73]:
df_to['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_to.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [74]:
df_to.shape

(103, 3)

In [75]:
print("Number of rows in df_to dataframe: ",df_to.shape)

Number of rows in df_to dataframe:  (103, 3)


# Part 2: 

In [76]:
torronto_geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
print("data loaded.")
torronto_geo_data.head()

data loaded.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [77]:
torronto_geo_data.columns = ['Postcode', 'Latitude', 'Longitude']
torronto_geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge two dataset 

In [81]:
df_tor = pd.merge(df_to,torronto_geo_data,on=['Postcode'],how='inner')
df_tor.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
